## Gömülü Temsiller

Önceki örneğimizde, `vocab_size` uzunluğunda yüksek boyutlu kelime torbası vektörleri üzerinde çalıştık ve düşük boyutlu konumsal temsil vektörlerinden seyrek tekil temsil (one-hot representation) vektörlerine açıkça dönüştürme yapıyorduk. Bu tekil temsil yöntemi bellek açısından verimli değildir, ayrıca her kelime birbirinden bağımsız olarak ele alınır, yani tekil kodlanmış vektörler kelimeler arasındaki herhangi bir anlamsal benzerliği ifade etmez.

Bu bölümde, **News AG** veri setini keşfetmeye devam edeceğiz. Başlamak için, verileri yükleyelim ve önceki not defterinden bazı tanımları alalım.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Gömme Nedir?

**Gömme** fikri, kelimeleri daha düşük boyutlu yoğun vektörlerle temsil etmektir; bu vektörler bir kelimenin anlamsal anlamını bir şekilde yansıtır. Daha sonra anlamlı kelime gömmelerinin nasıl oluşturulacağını tartışacağız, ancak şimdilik gömmeleri bir kelime vektörünün boyutunu düşürmenin bir yolu olarak düşünelim.

Bu nedenle, gömme katmanı bir kelimeyi girdi olarak alır ve belirli bir `embedding_size` boyutunda bir çıktı vektörü üretir. Bir anlamda, `Linear` katmanına çok benzer, ancak bir kelimeyi tekil kodlanmış bir vektör olarak almak yerine, kelime numarasını girdi olarak alabilir.

Ağımızda ilk katman olarak gömme katmanını kullanarak, kelime torbasından **gömme torbası** modeline geçebiliriz. Bu modelde, metnimizdeki her kelimeyi ilgili gömmeye dönüştürürüz ve ardından bu gömmeler üzerinde `sum`, `average` veya `max` gibi bir toplama fonksiyonu hesaplarız.

![Beş sıralı kelime için bir gömme sınıflandırıcıyı gösteren görsel.](../../../../../translated_images/tr/embedding-classifier-example.b77f021a7ee67eee.webp)

Sınıflandırıcı sinir ağımız, gömme katmanı ile başlayacak, ardından toplama katmanı ve en üstte doğrusal bir sınıflandırıcı ile devam edecektir:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Değişken Dizi Boyutuyla Çalışmak

Bu mimarinin bir sonucu olarak, ağımıza minibatch'ler belirli bir şekilde oluşturulmalıdır. Önceki bölümde, bag-of-words kullanırken, bir minibatch'teki tüm BoW tensörleri, metin dizimizin gerçek uzunluğundan bağımsız olarak eşit boyutta `vocab_size` idi. Kelime gömme yöntemine geçtiğimizde, her bir metin örneğinde değişken sayıda kelimeyle karşılaşırız ve bu örnekleri minibatch'lere birleştirirken bazı doldurma işlemleri uygulamamız gerekir.

Bu, veri kaynağına `collate_fn` fonksiyonunu sağlayarak yapılabilir:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Gömülü sınıflandırıcıyı eğitme

Artık uygun bir veri yükleyici tanımladığımıza göre, önceki bölümde tanımladığımız eğitim fonksiyonunu kullanarak modeli eğitebiliriz:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Not**: Burada yalnızca zaman açısından 25k kayıt (bir tam epoch'tan daha az) için eğitim yapıyoruz, ancak eğitime devam edebilir, birkaç epoch için eğitim yapmak üzere bir fonksiyon yazabilir ve daha yüksek doğruluk elde etmek için öğrenme oranı parametresiyle deneyler yapabilirsiniz. Yaklaşık %90 doğruluğa ulaşabilmelisiniz.


### EmbeddingBag Katmanı ve Değişken Uzunluklu Dizi Temsili

Önceki mimaride, dizileri bir minibatch'e sığdırmak için hepsini aynı uzunlukta doldurmamız gerekiyordu. Bu, değişken uzunluklu dizileri temsil etmenin en verimli yolu değildir - başka bir yaklaşım, tüm dizilerin bir büyük vektördeki offsetlerini tutan bir **offset** vektörü kullanmak olabilir.

![Offset dizi temsilini gösteren bir görüntü](../../../../../translated_images/tr/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Not**: Yukarıdaki resimde karakter dizisi gösteriyoruz, ancak örneğimizde kelime dizileriyle çalışıyoruz. Ancak, dizileri offset vektörüyle temsil etme genel prensibi aynı kalır.

Offset temsiliyle çalışmak için [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) katmanını kullanıyoruz. Bu katman `Embedding` katmanına benzer, ancak içerik vektörü ve offset vektörünü giriş olarak alır ve ayrıca `mean`, `sum` veya `max` olabilen bir ortalama katmanı içerir.

İşte `EmbeddingBag` kullanan değiştirilmiş bir ağ:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Veri kümesini eğitime hazırlamak için, offset vektörünü hazırlayacak bir dönüşüm fonksiyonu sağlamamız gerekiyor:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Önceki tüm örneklerden farklı olarak, ağımız artık iki parametre kabul ediyor: veri vektörü ve ofset vektörü, bunlar farklı boyutlardadır. Benzer şekilde, veri yükleyicimiz de bize 2 yerine 3 değer sağlıyor: hem metin hem de ofset vektörleri özellik olarak sağlanıyor. Bu nedenle, eğitim fonksiyonumuzu buna uygun şekilde biraz ayarlamamız gerekiyor:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Anlamsal Gömüler: Word2Vec

Önceki örneğimizde, modelin gömme katmanı kelimeleri vektör temsiline eşlemeyi öğrendi, ancak bu temsil çok fazla anlamsal anlam taşımıyordu. Benzer kelimelerin veya eş anlamlıların, belirli bir vektör mesafesi (örneğin, öklid mesafesi) açısından birbirine yakın olan vektörlere karşılık geldiği bir vektör temsili öğrenmek güzel olurdu.

Bunu yapmak için, gömme modelimizi büyük bir metin koleksiyonu üzerinde özel bir şekilde önceden eğitmemiz gerekiyor. Anlamsal gömüleri eğitmenin ilk yöntemlerinden biri [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) olarak adlandırılır. Bu yöntem, kelimelerin dağıtılmış bir temsilini üretmek için kullanılan iki ana mimariye dayanır:

 - **Sürekli kelime torbası** (CBoW) — Bu mimaride, modeli çevresel bağlamdan bir kelime tahmin etmeye eğitiriz. $$(W_{-2},W_{-1},W_0,W_1,W_2)$$ ngrami verildiğinde, modelin amacı $W_0$'ı $$(W_{-2},W_{-1},W_1,W_2)$$'den tahmin etmektir.
 - **Sürekli atlama-gramı** CBoW'un tersidir. Model, bağlam kelimelerinin çevresel penceresini kullanarak mevcut kelimeyi tahmin eder.

CBoW daha hızlıdır, ancak atlama-gramı daha yavaş olmasına rağmen nadir kelimeleri temsil etmede daha iyi bir iş çıkarır.

![Kelimeyi vektörlere dönüştürmek için hem CBoW hem de Skip-Gram algoritmalarını gösteren bir görüntü.](../../../../../translated_images/tr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News veri seti üzerinde önceden eğitilmiş word2vec gömüsünü denemek için **gensim** kütüphanesini kullanabiliriz. Aşağıda 'neural' kelimesine en benzer kelimeleri buluyoruz.

> **Not:** İlk kez kelime vektörleri oluşturduğunuzda, bunları indirmek biraz zaman alabilir!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kelimeden vektör gömmeleri de hesaplayabiliriz, sınıflandırma modeli eğitiminde kullanılmak üzere (açıklık için yalnızca vektörün ilk 20 bileşenini gösteriyoruz):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Semantik gömmelerle ilgili harika şey, vektör kodlamasını manipüle ederek anlamı değiştirebilmenizdir. Örneğin, *kral* ve *kadın* kelimelerine mümkün olduğunca yakın, *adam* kelimesinden ise olabildiğince uzak olan bir kelime bulmayı isteyebiliriz:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Hem CBoW hem de Skip-Grams, yalnızca yerel bağlamları dikkate aldıkları için "tahmin edici" gömülerdir. Word2Vec, global bağlamdan yararlanmaz.

**FastText**, Word2Vec'in üzerine inşa edilerek her kelime için ve kelime içindeki karakter n-gramları için vektör temsilleri öğrenir. Bu temsillerin değerleri, her eğitim adımında bir vektörde ortalanır. Bu, ön eğitim sırasında ek bir hesaplama yükü getirirken, kelime gömülerinin alt kelime bilgilerini kodlamasını sağlar.

Başka bir yöntem olan **GloVe**, eş oluşum matrisinin fikrinden yararlanır ve eş oluşum matrisini daha ifade edici ve doğrusal olmayan kelime vektörlerine ayırmak için sinirsel yöntemler kullanır.

Gensim, birkaç farklı kelime gömme modelini desteklediği için gömüleri FastText ve GloVe olarak değiştirerek örnekle oynayabilirsiniz.


## PyTorch'ta Önceden Eğitilmiş Gömülüleri Kullanma

Yukarıdaki örneği, gömme katmanımızdaki matrisi Word2Vec gibi anlamsal gömülerle önceden dolduracak şekilde değiştirebiliriz. Önceden eğitilmiş gömülerin ve metin korpusumuzun kelime dağarcıklarının muhtemelen eşleşmeyeceğini göz önünde bulundurmamız gerekiyor, bu yüzden eksik kelimeler için ağırlıkları rastgele değerlerle başlatacağız:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Şimdi modelimizi eğitelim. Modeli eğitmek için gereken sürenin, daha büyük gömme katmanı boyutu ve dolayısıyla çok daha yüksek parametre sayısı nedeniyle önceki örneğe kıyasla önemli ölçüde daha uzun olduğunu unutmayın. Ayrıca, bu nedenle aşırı öğrenmeyi önlemek istiyorsak modelimizi daha fazla örnek üzerinde eğitmemiz gerekebilir.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Bizim durumumuzda, doğrulukta büyük bir artış görmüyoruz, bu muhtemelen oldukça farklı kelime dağarcıklarından kaynaklanıyor. 
Farklı kelime dağarcığı sorununu aşmak için aşağıdaki çözümlerden birini kullanabiliriz:
* Kelime dağarcığımız üzerinde word2vec modelini yeniden eğitmek
* Veri setimizi önceden eğitilmiş word2vec modelinin kelime dağarcığı ile yüklemek. Veri setini yüklemek için kullanılan kelime dağarcığı yükleme sırasında belirtilebilir.

İkinci yaklaşım daha kolay görünüyor, özellikle de PyTorch `torchtext` framework'ünün gömülü destek içerdiği için. Örneğin, GloVe tabanlı bir kelime dağarcığını şu şekilde oluşturabiliriz:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Yüklenen kelime dağarcığı aşağıdaki temel işlemleri içerir:
* `vocab.stoi` sözlüğü, bir kelimeyi sözlük indeksine dönüştürmemizi sağlar
* `vocab.itos` bunun tersini yapar - bir sayıyı kelimeye dönüştürür
* `vocab.vectors`, gömme vektörlerinin dizisidir, bu nedenle bir kelimenin `s` gömme vektörünü almak için `vocab.vectors[vocab.stoi[s]]` kullanmamız gerekir

İşte **kind-man+woman = queen** denklemini göstermek için gömme vektörleriyle yapılan bir manipülasyon örneği (çalışması için katsayıyı biraz ayarlamam gerekti):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Bu gömüler kullanılarak sınıflandırıcıyı eğitmek için, öncelikle veri setimizi GloVe kelime dağarcığını kullanarak kodlamamız gerekiyor:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Yukarıda gördüğümüz gibi, tüm vektör gömmeleri `vocab.vectors` matrisinde saklanır. Bu, bu ağırlıkları gömme katmanının ağırlıklarına basit bir kopyalama ile yüklemeyi oldukça kolaylaştırır:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Şimdi modelimizi eğitelim ve daha iyi sonuçlar alıp almadığımıza bakalım:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Veri setimizdeki bazı kelimelerin önceden eğitilmiş GloVe sözlüğünde bulunmaması ve dolayısıyla esasen göz ardı edilmesi, doğrulukta önemli bir artış görmememizin nedenlerinden biridir. Bu durumu aşmak için, kendi veri setimiz üzerinde kendi gömme yöntemlerimizi eğitebiliriz.


## Bağlamsal Gömüler

Word2Vec gibi geleneksel önceden eğitilmiş gömü temsillerinin en önemli sınırlamalarından biri, kelime anlamı ayrımının zorluğudur. Önceden eğitilmiş gömüler, kelimelerin bağlamdaki anlamlarının bir kısmını yakalayabilse de, bir kelimenin tüm olası anlamları aynı gömüye kodlanır. Bu durum, 'play' gibi birçok kelimenin kullanıldıkları bağlama bağlı olarak farklı anlamlara sahip olması nedeniyle, sonraki modellerde sorunlara yol açabilir.

Örneğin, 'play' kelimesi şu iki farklı cümlede oldukça farklı anlamlara sahiptir:
- Tiyatroda bir **oyuna** gittim.
- John arkadaşlarıyla **oyun** oynamak istiyor.

Yukarıdaki önceden eğitilmiş gömüler, 'play' kelimesinin her iki anlamını da aynı gömüde temsil eder. Bu sınırlamayı aşmak için, kelimelerin farklı bağlamlarda nasıl bir araya gelebileceğini *bilen* ve büyük bir metin derlemi üzerinde eğitilmiş olan **dil modeli** temelinde gömüler oluşturmamız gerekir. Bağlamsal gömüleri tartışmak bu eğitimin kapsamı dışında olsa da, bir sonraki bölümde dil modellerini ele alırken bu konuya geri döneceğiz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalardan sorumlu değiliz.
